# Tensorflow 前端

参考: [TVM Tensorflow 前端](https://xinetzone.github.io/tvm/docs/arch/frontend/tensorflow.html)

下面以 [mobilenet_v2 float_v2_1.4_224](https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.4_224.tgz) 为例，展示 Tensorflow 前端。

先运行简单的测试：

In [1]:
import numpy as np
import tensorflow as tf
try:
    tf1 = tf.compat.v1
except (ImportError, AttributeError):
    tf1 = tf
import tvm.relay.testing.tf as tf_testing
import tvm
from tvm import relay
from tvm.contrib import graph_executor


def convert_to_list(x):
    if not isinstance(x, list):
        x = [x]
    return x

def run_tf_graph(sess, input_data, input_node, output_node):
    """执行 TensorFlow 的通用函数。"""
    input_data = convert_to_list(input_data)
    input_node = convert_to_list(input_node)
    output_node = convert_to_list(output_node)

    tensor = [sess.graph.get_tensor_by_name(output_name) for output_name in output_node]

    input_dict = {e: input_data[i] for i, e in enumerate(input_node)}
    if len(input_node) == 1 and input_node[0] == "":
        output_data = sess.run(tensor)
    else:
        output_data = sess.run(tensor, input_dict)
    return output_data

2023-06-12 14:49:19.892229: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 14:49:19.940377: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 14:49:19.941288: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 14:49:21.295097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
shape = 1, 224, 224, 3
data = np.random.uniform(size=shape).astype("float32")
out_node = "MobilenetV2/Predictions/Reshape_1"
input_name = "input"
with tf.Graph().as_default():
    graph_def = tf_testing.get_workload(
        "https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.4_224.tgz",
        "mobilenet_v2_1.4_224_frozen.pb",
    )
    # 调用实用程序将图定义导入默认 graph
    graph_def = tf_testing.ProcessGraphDefParam(graph_def)
    with tf1.Session() as sess:
        # 添加 shapes 到 graph
        graph_def = tf_testing.AddShapesToGraphDef(sess, out_node)
        tf_output = run_tf_graph(sess, data, f"{input_name}:0", f"{out_node}:0")
        mod, params = relay.frontend.from_tensorflow(
            graph_def,
            shape={
                input_name: shape
            }
        )

Instructions for updating:
Use tf.gfile.GFile.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


2023-06-12 14:49:27.250575: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-12 14:49:27.533544: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


In [3]:
desired_layouts = {
    # 'image.resize2d': ['NCHW'],
    'nn.conv2d': ['NCHW', 'default'],
    'nn.max_pool2d': ['NCHW', 'default'],
    'nn.avg_pool2d': ['NCHW', 'default'],
}

# 将布局转换为 NCHW
# RemoveUnusedFunctions 用于清理图。
seq = tvm.transform.Sequential([relay.transform.RemoveUnusedFunctions(),
                                relay.transform.ConvertLayout(desired_layouts)])
with tvm.transform.PassContext(opt_level=3):
    mod = seq(mod)

In [4]:
target = tvm.target.Target("llvm", host="llvm")
dev = tvm.cpu(0)
with relay.build_config(opt_level=3):
     lib = relay.build(mod, target, params=params)
m = graph_executor.GraphModule(lib["default"](dev))
m.set_input(**{input_name: data})
m.run()
tvm_output = [m.get_output(kk).numpy() for kk in range(m.get_num_outputs())]
np.testing.assert_allclose(
    np.squeeze(tvm_output[0]), np.squeeze(tf_output[0]), rtol=1e-5, atol=1e-5
)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


## ImageNet 测试

使用 [slim/datasets/imagenet.py](https://github.com/tensorflow/models/blob/master/research/slim/datasets/imagenet.py) 获取数据标签信息。

In [5]:
from github import Github


g = Github(user_agent="xinetzone")
repo = g.get_repo("tensorflow/models")
label_content = repo.get_contents("research/slim/datasets/imagenet_lsvrc_2015_synsets.txt")
imagenet_labels = label_content.decoded_content.decode().split()
assert len(imagenet_labels) == 1000
metadata = repo.get_contents("research/slim/datasets/imagenet_metadata.txt")
imagenet_metadata = metadata.decoded_content.decode().splitlines()
synset_to_human = {}
for metadata in imagenet_metadata:
    name, value = metadata.split("\t")
    synset_to_human[name] = value

TensorFlow 中将背景作为标签 `0`，故而，有：

In [6]:
name2id = {name: k+1 for k, name in enumerate(imagenet_labels)}

读取数据集：

In [7]:
from __future__ import annotations
from pathlib import Path
from dataclasses import dataclass
from PIL import Image
import numpy as np


@dataclass
class ImageNet:
    root: str | Path # 数据根目录
    height: int = 224
    width: int = 224

    def __post_init__(self):
        self.root = Path(self.root)

    def data_iter(self, mode="train"):
        for root in (self.root/mode).iterdir():
            for path in root.iterdir():
                with Image.open(path) as im:
                    if im.mode != "RGB":
                        im = im.convert("RGB")
                    img = np.array(im.resize((self.height, self.width)))
                yield img, root.name

In [8]:
from tqdm import tqdm

data_dir = "/media/pc/data/lxw/home/data/datasets/ILSVRC"
dataset = ImageNet(data_dir)

m = graph_executor.GraphModule(lib["default"](dev))
true_labels = []
pred_labels = []
for k, (img, label) in tqdm(enumerate(dataset.data_iter("val"))):
    img = img/128 - 1
    m.set_input(**{"input": np.expand_dims(img, 0)})
    m.run()
    tvm_output = m.get_output(0).numpy()
    pred_labels.append(tvm_output.argmax())
    true_labels.append(name2id[label])

50000it [16:28, 50.59it/s]


In [11]:
sum(np.array(true_labels) == np.array(pred_labels))/len(true_labels)

0.73074